In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and tokenizer
model_path = "/kaggle/input/llama-3.2/transformers/1b/1"  # Update with your model path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)



In [41]:
import pandas as pd 


df=pd.read_csv('/kaggle/input/myrcmdata/myRCM.tsv', sep='\t')
df.head()

,CVE_ID,Description,CWE
0,CVE-2024-0001,A condition exists in FlashArray Purity whereb...,CWE-1188
1,CVE-2024-0002,A condition exists in FlashArray Purity whereb...,NVD-CWE-noinfo
2,CVE-2024-0003,A condition exists in FlashArray Purity whereb...,NVD-CWE-noinfo
3,CVE-2024-0004,A condition exists in FlashArray Purity whereb...,CWE-94
4,CVE-2024-0005,A condition exists in FlashArray and FlashBlad...,CWE-77


In [29]:
import re
import torch

def generate_gt(cve_description):
    """Generates the CWE mapping for a given CVE description using LLaMA."""

    try:
        # Assuming `model`, `tokenizer`, and `device` are defined and model is loaded elsewhere
        # Set the pad_token if not already set
        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id  # Use eos_token_id as pad_token_id

        # Ensure the model is using the device
        model.to(device)

        # The obligatory prompt
        prompt = (f"Analyze the following CVE description and determine the appropriate CWE. "
                  f"Provide a justification. The last part of your response should contain only the CWE ID number.\n\n"
                  f"CVE Description: {cve_description}\n")

        # Tokenize the prompt and move inputs to the correct device
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

        # Set the pad_token_id for the model
        model.config.pad_token_id = tokenizer.pad_token_id

        # Generate response with a dynamic length limit
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=min(2000, inputs["input_ids"].shape[1] + 50),  # Be conservative with length
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id
        )

        # Decode the generated text
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        # Improved CWE ID extraction
        cwe_id_match = re.search(r'CWE-\d+', response)
        cwe_id = cwe_id_match.group(0) if cwe_id_match else "CWE ID not found"

        return {
            "full_response": response,
            "cwe_id": cwe_id
        }

    except Exception as e:
        print(f"Error generating CWE mapping: {str(e)}")
        return None

In [42]:
import pandas as pd
import random

# Assume `df` is your DataFrame containing the CVE descriptions
# Sample 200 random rows from the DataFrame
sampled_df = df.sample(n=200, random_state=42)  # random_state for reproducibility

# Initialize lists to store results
justifications = []
cwe_ids = []

# Iterate over the sampled rows and generate CWE mappings
for index, row in sampled_df.iterrows():
    cve_description = row["Description"]
    result = generate_gt(cve_description)  # Assuming this function is defined elsewhere
    if result:
        justifications.append(result['full_response'])
        cwe_ids.append(result['cwe_id'])
    else:
        justifications.append("Error in processing")
        cwe_ids.append("Error")

# Create a DataFrame to store and display the results
results_df = pd.DataFrame({
    "Description": sampled_df["Description"].reset_index(drop=True),
    "Justification": justifications,
    "CWE_ID": cwe_ids
})



# Optionally, save results to a CSV for further analysis
results_df.to_csv("resultsllama.tsv", index=False)


In [43]:
# Optionally, save results to a CSV for further analysis
results_df.to_csv("resultsllama.csv", index=False)

In [44]:
res=pd.read_csv("/kaggle/working/resultsllama.csv")
res.head()

,Description,Justification,CWE_ID
0,The WordPress Review & Structure Data Schema P...,Analyze the following CVE description and dete...,CWE-79
1,The LearnDash LMS plugin for WordPress is vuln...,Analyze the following CVE description and dete...,CWE ID not found
2,A flaw was found in the decompression function...,Analyze the following CVE description and dete...,CWE ID not found
3,"The 'HTML5 Video Player' WordPress Plugin, ver...",Analyze the following CVE description and dete...,CWE-79
4,Privilege escalation vulnerability in Lamassu ...,Analyze the following CVE description and dete...,CWE-119


In [45]:
merged_df = pd.merge(df, res, on='Description', how='inner') 
merged_df

,CVE_ID,Description,CWE,Justification,CWE_ID
0,CVE-2024-0089,NVIDIA GPU Display Driver for Windows contains...,CWE-665,Analyze the following CVE description and dete...,CWE ID not found
1,CVE-2024-0113,"NVIDIA Mellanox OS, ONYX, Skyway, and MetroX-3...",CWE-22,Analyze the following CVE description and dete...,CWE-79
2,CVE-2024-0123,NVIDIA CUDA toolkit for Windows and Linux cont...,CWE-1285,Analyze the following CVE description and dete...,CWE-400
3,CVE-2024-0154,Dell PowerEdge Server BIOS and Dell Precision ...,CWE-788,Analyze the following CVE description and dete...,CWE-119
4,CVE-2024-0163,Dell PowerEdge Server BIOS and Dell Precision ...,CWE-367,Analyze the following CVE description and dete...,CWE-119
...,...,...,...,...,...
198,CVE-2024-20088,"In keyinstall, there is a possible out of boun...",CWE-125,Analyze the following CVE description and dete...,CWE ID not found
199,CVE-2024-20089,"In wlan, there is a possible denial of service...",CWE-754,Analyze the following CVE description and dete...,CWE ID not found
200,CVE-2024-20093,"In vdec, there is a possible out of bounds rea...",CWE-125,Analyze the following CVE description and dete...,CWE ID not found
201,CVE-2024-2011,A heap-based buffer overflow vulnerability exi...,CWE-787,Analyze the following CVE description and dete...,CWE-200


In [46]:
import pandas as pd

def compute_rcm_accuracy(df, col):
   
    correct = 0
    total = 0

    for idx, row in df.iterrows():
        pred = row[col].upper()  # Convert prediction to uppercase for case-insensitive comparison
        #gt = str(row['GT']).upper() # Convert Ground Truth to upper case and string to avoid potential type errors
        gt = str(row['CWE']).upper()

        # Validate if the prediction starts with 'CWE-'
        if pred.startswith('CWE-'):
            total += 1
            if pred == gt:
                correct += 1
        else:
            print(f'Invalid response at row {idx + 1}: {pred}') # Include the invalid prediction for debugging

    # Calculate and return the accuracy, handling potential division by zero
    accuracy = (correct / total) * 100 if total > 0 else 0  
    return accuracy

In [47]:
# prompt: PRINT THE ACCURACY

accuracy = compute_rcm_accuracy(merged_df, 'CWE_ID')
print(f"Accuracy: {accuracy:.2f}%")


Invalid response at row 1: CWE ID NOT FOUND
Invalid response at row 8: CWE ID NOT FOUND
Invalid response at row 10: CWE ID NOT FOUND
Invalid response at row 11: CWE ID NOT FOUND
Invalid response at row 12: CWE ID NOT FOUND
Invalid response at row 17: CWE ID NOT FOUND
Invalid response at row 22: CWE ID NOT FOUND
Invalid response at row 23: CWE ID NOT FOUND
Invalid response at row 25: CWE ID NOT FOUND
Invalid response at row 27: CWE ID NOT FOUND
Invalid response at row 28: CWE ID NOT FOUND
Invalid response at row 29: CWE ID NOT FOUND
Invalid response at row 32: CWE ID NOT FOUND
Invalid response at row 33: CWE ID NOT FOUND
Invalid response at row 34: CWE ID NOT FOUND
Invalid response at row 36: CWE ID NOT FOUND
Invalid response at row 39: CWE ID NOT FOUND
Invalid response at row 40: CWE ID NOT FOUND
Invalid response at row 43: CWE ID NOT FOUND
Invalid response at row 44: CWE ID NOT FOUND
Invalid response at row 45: CWE ID NOT FOUND
Invalid response at row 46: CWE ID NOT FOUND
Invalid resp